# model.predict()
## 🧠 一、model.predict() 是什么？
在 TensorFlow / Keras 中，model.predict() 是用于**模型推理（inference）**的接口 —— 即用训练好的模型，对新数据进行预测。

## 🔧 二、model.predict() 的函数定义

In [ ]:
model.predict(
    x,
    batch_size=None,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False
)


| 参数名                   | 说明                                              |
| --------------------- | ----------------------------------------------- |
| `x`                   | 输入数据，支持 Numpy 数组、Tensor、`tf.data.Dataset`、生成器等。 |
| `batch_size`          | 每批输入数据的大小。默认由模型自动决定。                            |
| `verbose`             | 是否显示进度条（0 静默，1 显示进度）                            |
| `steps`               | 如果 `x` 是生成器或 Dataset，表示要预测的总步骤数                 |
| `callbacks`           | 预测时调用的回调函数（一般用于高级用途）                            |
| `use_multiprocessing` | 是否使用多进程生成批次数据（配合 `workers` 参数）                  |
| `workers`             | 并行线程数，用于生成数据                                    |


### 🧪 三、基本使用示例
示例：回归问题（输出连续值）

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 构造简单数据
X = np.array([[0], [1], [2], [3], [4]])
y = np.array([[0], [1], [2], [3], [4]])

# 构建模型
model = Sequential([
    Dense(10, activation='relu', input_shape=(1,)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

# 预测
X_test = np.array([[1.5], [2.5], [3.5]])
pred = model.predict(X_test)
print(pred)


### 🎯 四、分类任务中 predict 的使用
✅ 二分类（sigmoid 输出）

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# 数据
X = np.random.rand(100, 5)
y = (X[:, 0] + X[:, 1] > 1).astype(int)

model = Sequential([
    Dense(8, activation='relu', input_shape=(5,)),
    Dense(1, activation='sigmoid')
])

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=0)

# 预测
probs = model.predict(X[:5])
labels = (probs > 0.5).astype(int)
print("概率：", probs)
print("预测标签：", labels)


✅ 多分类（softmax 输出）

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Softmax
import numpy as np

X = np.random.rand(100, 10)
y = np.random.randint(0, 3, size=100)  # 3 类
y_cat = to_categorical(y, num_classes=3)

model = Sequential([
    Dense(32, activation='relu', input_shape=(10,)),
    Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y_cat, epochs=10, verbose=0)

# 预测
pred_probs = model.predict(X[:5])
pred_labels = np.argmax(pred_probs, axis=1)
print("概率输出：\n", pred_probs)
print("预测类别：", pred_labels)


## 🧩 五、输入数据类型支持
| 输入类型              | 是否支持 | 说明                        |
| ----------------- | ---- | ------------------------- |
| Numpy 数组          | ✅    | 最常见                       |
| TensorFlow Tensor | ✅    | 用 `tf.constant()` 或其他创建方式 |
| `tf.data.Dataset` | ✅    | 推荐用于大数据集                  |
| Python 生成器        | ✅    | 自定义数据生成逻辑时使用              |

## 🧠 六、模型预测原理：本质 = 前向传播
model.predict() 的核心是调用模型的 前向传播逻辑

等价于：

outputs = model(x_input, training=False)

所以你也可以直接调用：

outputs = model(tf.constant(X_test), training=False)

但推荐使用 model.predict()，因为它封装了批处理、并发、安全性、更方便。
### 🔍 七、model.predict() vs model.__call__() vs model(x)
| 方法                         | 用途             | 是否启用训练行为           | 是否常用        |
| -------------------------- | -------------- | ------------------ | ----------- |
| `model.predict()`          | 正式预测           | `training=False`   | ✅ 常用        |
| `model(x)`                 | 前向推理或训练        | 默认 `training=True` | ✅ 在自定义训练中常用 |
| `model(x, training=False)` | 控制是否训练行为       | 手动设置               | ✅ 自定义最灵活    |
| `model.__call__()`         | 等价于 `model(x)` | 不建议手动使用            | ❌ 少用        |

## ⚠️ 八、注意事项
1、输入 shape 要正确：单个样本必须 reshape 成 (1, 特征维度)，否则可能会报错。\
2、批量预测更高效： 用 batch_size 控制预测批次大小（尤其是数据集较大时）。\
3、模型必须已训练： 调用 .predict() 之前应先 .fit() 或 .load_weights()。\
4、不要用于训练时评估指标： 训练期间的评估应用 model.evaluate()，预测结果的评价需你手动计算准确率、F1 等。

### ✅ 九、快速回顾总结
| 情况         | 推荐方式                                  |
| ---------- | ------------------------------------- |
| 新样本预测      | `model.predict(x)`                    |
| 要取最大概率类别标签 | `np.argmax(model.predict(x), axis=1)` |
| 要输出分类概率    | `model.predict(x)`                    |
| 高效批量预测     | 设置 `batch_size`                       |
| 大规模数据      | 使用 `tf.data.Dataset`                  |
